In [1]:
!pip install langchain langchain_openai langchain_community transformers langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 583.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.3 MB/s eta 0:00:00


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "key"

In [3]:
# 구글 드라이브 연결(데이터 로드를 위해서)
try:
    from google.colab import drive

    drive.mount('/content/')
    DATA_PATH = "/content/"
except:
    DATA_PATH = "/content/"

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 3.5 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 리스트
pdf_files = [
    "2024년 하반기 3급 신입사원 채용 직무소개서.pdf",
    "삼성전자 DS부문 24하 공채 직무기술서.pdf"
]

# 모든 문서 로드
docs = []
for path in pdf_files:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# 총 로드된 문서 수 확인
print("총 문서 수:", len(docs))


총 문서 수: 188


In [6]:
docs[0]

Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2024-09-04T13:46:13+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'admin', 'moddate': '2024-09-04T13:46:13+09:00', 'source': '2024년 하반기 3급 신입사원 채용 직무소개서.pdf', 'total_pages': 84, 'page': 0, 'page_label': '1'}, page_content='')

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=0)

In [8]:
split_doc1 = text_splitter.split_documents(docs)
len(split_doc1)

374

In [9]:
split_doc1[0]

Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2024-09-04T13:46:13+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'admin', 'moddate': '2024-09-04T13:46:13+09:00', 'source': '2024년 하반기 3급 신입사원 채용 직무소개서.pdf', 'total_pages': 84, 'page': 1, 'page_label': '2'}, page_content='회로개발\n회로 기술을 기반으로 삼성전자 제품 및 솔루션의 혁신적인 가치를 창출합니다')

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [11]:
# 임베딩 차원 크기를 계산
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1536


In [12]:
from langchain_chroma import Chroma

# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
db = Chroma.from_documents(
    split_doc1, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name="my_db"
)

In [ ]:
# 저장된 데이터 확인
db.get()

{'ids': ['aba66074-c46d-4615-bfd8-bb7442e56a9f',
  '0add3427-9e6b-4460-8d03-296f1215a5be',
  '46758b29-2a5f-4192-9af0-3df7683b4e7c',
  'dab26176-54c6-4c3e-acd3-40cc93742e57',
  '1062a52e-05d3-4a1f-b9b2-1f06fcd7831e',
  'fe549197-18fe-462f-aec1-57657829b8a6',
  'd4381a1d-7915-42c2-8806-6aef6d92690d',
  'bd521ee2-6a88-4934-9ccd-a427e28bd0c8',
  '3f1f374c-8fd2-4bfd-a7e6-e2d8cdab16e0',
  'd66399e6-24b2-41b5-aeb5-e8e5d690ecdf',
  '3dba92bb-b23e-4ab1-abe9-7842c2f8a2b1',
  'd3a164d3-49ab-4181-9a40-d535e9359281',
  'baafa5c9-d63a-49a0-ad57-50e16edf4878',
  '83490274-4e2d-4491-a297-ca918e0dd7e3',
  '5c2d8f57-f5a0-47dc-8a7b-81bd6d0f3723',
  '05cb6bcb-3689-4d8c-a19a-59cdd48526fd',
  '0d73f50c-1ffc-4a0b-aef9-8dbece1f78a5',
  '5d4b9054-888e-47fd-b841-e5fd23958930',
  '65e2594f-3c08-42a2-88ff-72aeac209766',
  '32796b50-78d1-4527-8e93-e3de524fa180',
  '304d89a5-81bf-473c-b8e8-fbb30a67fb2c',
  '5df575eb-0eb2-4c87-ae95-d1c4cedddadb',
  '5a3b2a70-0c9a-44cd-a63d-4236c5fa0b27',
  'b8982f89-a48f-4abb-8b20-

In [ ]:
db = Chroma.from_documents(
    split_doc1, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name="search_db"
)

In [ ]:
db.get()

{'ids': ['b88b8b20-7bbc-4405-a61a-3fc0de5869be',
  'c257f842-cfe1-42bd-890b-0e3aac18162d',
  '20721bc4-9828-4bcd-a247-c3faf4726f12',
  '4fd64ffe-c87f-483b-a55e-ce0a58ced7f1',
  '403feecc-f21f-46e2-bfcd-7f93cf023724',
  'fc616651-553e-427b-bed8-82d1460793fd',
  'd82dc870-699d-4530-9ec9-289290b6a06b',
  '73155953-8126-4d80-b66e-ac0dda522d26',
  '5ac701b7-3819-4185-b89d-a552755931bc',
  '2e7d39b8-fac8-4a06-9744-d0f6096e4b06',
  'f61f1a9d-eb58-47ea-b54b-d468063083af',
  '5266923b-2329-4ac5-9909-10e7ee8ec845',
  'f877e6e6-b779-45c3-a294-1343ca46c8ff',
  '0cc2537d-42cb-4ff8-83a1-f06e5c9068e5',
  'd5482371-fa80-4170-8f8a-5eb37f2a92b5',
  'fb2e120a-c540-408c-bc43-b33515f4c0cc',
  '004f88b2-b5a8-4d62-89c0-267104644b03',
  '63107fb7-6576-4433-8bc7-0708dff99bb4',
  'c1ba28a1-dfe2-4acc-b46a-96e529c12857',
  '75409afe-2b44-4e08-9a98-3dfda60ec3a7',
  '33fd34e5-4721-46ef-a06b-0a14832ad3ca',
  '515a5c75-3517-4a4a-b954-8390493c1134',
  'f91df458-4042-4f92-b462-84b9dcdc6c2e',
  '81f3a29e-0cb7-4631-b8ec-

In [ ]:
# 유사도 검색
result = db.similarity_search("TF IDF 에 대하여 알려줘")
len(result)

In [ ]:
result[0].page_content

In [ ]:
result[1].page_content

In [ ]:
# k 값 지정
result = db.similarity_search("TF IDF 에 대하여 알려줘", k=2)
len(result)

In [ ]:
result

In [ ]:
# filter 사용
result = db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": DATA_PATH+"nlp-keywords.txt"}, k=2
)
len(result)

In [ ]:
result

In [ ]:
result = db.max_marginal_relevance_search(
    "TF IDF 에 대하여 알려줘", k=4, fetch_k=10)
len(result)

In [ ]:
result